In [2]:
import numpy as np
import os

import skimage.io
import skimage.viewer
from skimage.io import imread
from skimage.transform import rescale, resize

from scipy import ndimage as ndi
from skimage import feature

from skimage.transform import rotate, AffineTransform
from skimage import transform

from skimage.util import random_noise
import random
import datetime

/var/folders/mz/vg384l6j0h542rj14nk8lhkh0000gn/T/ipykernel_2158/3364474064.py:5: UserWarning: viewer requires Qt
  import skimage.viewer


In [3]:
cnn = 0
img_nums = 15

In [4]:
import numpy as np

np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
np.random.seed(30)

In [5]:
print(tf.__version__)

2.10.0


In [6]:
# 0: Left Swipe, 1: Right Swipe, 2: Stop, 3: Thumbs Down, 4: Thumbs Up
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())
train_len = len(train_doc)
val_len = len(val_doc)
train_dirs = []
val_dirs = []
for i in range(train_doc.shape[0]):
    train_dir = train_doc[i].split(';')[0]
    if(i < 100):
        val_dir = val_doc[i].split(';')[0]
        val_dirs.append(val_dir)
        train_dirs.append(train_dir)
batch_size = 90                                        #experiment with the batch size
print(f'Batch Size = {batch_size}, \nTrain length = {len(train_dirs)}, \nValidation length = {len(val_dirs)}')
print(f'Training Length = {train_len}, Validation Length = {val_len}')

Batch Size = 90, 
Train length = 100, 
Validation length = 100
Training Length = 663, Validation Length = 100


In [7]:
def generator(source_path, folder_list, batch_size, lenth, aug_aff = 0, aug_edg = 0, aug_nois = 0):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    if(img_nums == 30):
        img_idx = range(0, 30)                                                                     #create a list of image numbers you want to use for a particular video
    else:
        img_idx = range(0, 30, 2)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = lenth // batch_size                                                     # calculate the number of batches
        num_imgs = lenth - num_batches*batch_size
        for batch in range(num_batches):                                                          # we iterate over the number of batches
            x = len(img_idx)
            y = 96
            z = 96
            batch_data = np.zeros((batch_size,x,y,z,3))                                           # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5))                                               # batch_labels is the one hot representation of the output
            if(aug_aff):
                aug_batch_data = np.zeros((batch_size,x,y,z,3))
                aug_batch_labels = np.zeros((batch_size,5))
            if(aug_nois):
                nois_batch_data = np.zeros((batch_size,x,y,z,3))
                nois_batch_labels = np.zeros((batch_size,5))
            if(aug_edg):
                edg_batch_data = np.zeros((batch_size,x,y,z,3))
                edg_batch_labels = np.zeros((batch_size,5))



            for folder in range(batch_size):                                                      # iterate over the batch_size
                imgs = os.listdir('/Users/sharadchandra/Hand_Gestures/train'+'/'+ t[folder + (batch*batch_size)].split(';')[0])  # read all the images in the folder
                for idx,item in enumerate(img_idx):                                               #  Iterate over the frames/images of a folder to read them in
                    image = imread('/Users/sharadchandra/Hand_Gestures/train'+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    image = resize(image = image, output_shape=(y, z))

                    if(aug_edg):
                        img1 = image.astype('float64')
                        img1[:, :, 0] += 0.2 * np.random.random(img1[:, :, 0].shape)
                        img1[:, :, 1] += 0.2 * np.random.random(img1[:, :, 1].shape)
                        img1[:, :, 2] += 0.2 * np.random.random(img1[:, :, 2].shape)
                        edges10 = feature.canny(img1[:, :, 0])
                        edges20 = feature.canny(img1[:, :, 0], sigma=3)
                        edges11 = feature.canny(img1[:, :, 1])
                        edges21 = feature.canny(img1[:, :, 1], sigma=3)
                        edges12 = feature.canny(img1[:, :, 2])
                        edges22 = feature.canny(img1[:, :, 2], sigma=3)
                        edg_batch_data[folder, idx, :, :, 0] = edges20
                        edg_batch_data[folder, idx, :, :, 1] = edges21
                        edg_batch_data[folder, idx, :, :, 2] = edges22

                    
                    if(aug_nois):
                        image_noise_aug = random_noise(image, var = 0.1)
                        nois_batch_data[folder, idx, :, :, 0] = image_noise_aug[:, :, 0]
                        nois_batch_data[folder, idx, :, :, 1] = image_noise_aug[:, :, 1]
                        nois_batch_data[folder, idx, :, :, 2] = image_noise_aug[:, :, 2]


                    image[:, :, 0] = (image [:, :, 0] - np.min(image[:, :, 0]))/(np.max(image[:, :, 0]) - np.min(image[:, :, 0]))
                    image[:, :, 1] = (image [:, :, 1] - np.min(image[:, :, 1]))/(np.max(image[:, :, 1]) - np.min(image[:, :, 1]))
                    image[:, :, 2] = (image [:, :, 2] - np.min(image[:, :, 2]))/(np.max(image[:, :, 2]) - np.min(image[:, :, 2]))

                    if(aug_aff):
                        shear_val = random.uniform(0.0, -0.5)
                        tf = AffineTransform(shear = shear_val)
                        img_aff = transform.warp(image, tf, order=1, preserve_range=True, mode='wrap')
                        aug_batch_data[folder, idx, :, :, 0] = img_aff[:, :, 0]
                        aug_batch_data[folder, idx, :, :, 1] = img_aff[:, :, 1]
                        aug_batch_data[folder, idx, :, :, 2] = img_aff[:, :, 2]

                    batch_data[folder,idx,:,:,0] = image[:, :, 0]       #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:, :, 1]       #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:, :, 2]       #normalise and feed in the image


                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                if aug_aff:
                    aug_batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

                if aug_nois:
                    nois_batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                    
                if aug_edg:
                    edg_batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

            if aug_aff:
                batch_data = np.append(batch_data, aug_batch_data, axis = 0)
                batch_labels = np.append(batch_labels, aug_batch_labels, axis = 0) 

            if aug_nois:
                batch_data = np.append(batch_data, nois_batch_data, axis = 0) 
                batch_labels = np.append(batch_labels, nois_batch_labels, axis = 0)
                
            if aug_edg:
                batch_data = np.append(batch_data, edg_batch_data, axis = 0) 
                batch_labels = np.append(batch_labels, edg_batch_labels, axis = 0) 



            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        x = len(img_idx)
        y = 96
        z = 96
        batch_data = np.zeros((num_imgs,x,y,z,3))                                               # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((num_imgs,5))

        if(aug_aff):
            aug_batch_data = np.zeros((num_imgs,x,y,z,3))
            aug_batch_labels = np.zeros((num_imgs,5))
        if(aug_nois):
            nois_batch_data = np.zeros((num_imgs,x,y,z,3))
            nois_batch_labels = np.zeros((num_imgs,5))
        if(aug_edg):
            edg_batch_data = np.zeros((num_imgs,x,y,z,3))
            edg_batch_labels = np.zeros((num_imgs,5))

        for folder in range(num_imgs):
          imgs = os.listdir('/Users/sharadchandra/Hand_Gestures/train'+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
          for idx, item in enumerate(img_idx):
            image = imread('/Users/sharadchandra/Hand_Gestures/train'+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes

            image = resize(image = image, output_shape=(y, z))

            if(aug_edg):
                img1 = image.astype('float64')
                img1[:, :, 1] += 0.2 * np.random.random(img1[:, :, 1].shape)
                img1[:, :, 2] += 0.2 * np.random.random(img1[:, :, 2].shape)
                edges10 = feature.canny(img1[:, :, 0])
                edges20 = feature.canny(img1[:, :, 0], sigma=3)
                edges11 = feature.canny(img1[:, :, 1])
                edges21 = feature.canny(img1[:, :, 1], sigma=3)
                edges12 = feature.canny(img1[:, :, 2])
                edges22 = feature.canny(img1[:, :, 2], sigma=3)
                edg_batch_data[folder, idx, :, :, 0] = edges20
                edg_batch_data[folder, idx, :, :, 1] = edges21
                edg_batch_data[folder, idx, :, :, 2] = edges22

                    
            if(aug_nois):
                image_noise_aug = random_noise(image, var = 0.1)
                nois_batch_data[folder, idx, :, :, 0] = image_noise_aug[:, :, 0]
                nois_batch_data[folder, idx, :, :, 1] = image_noise_aug[:, :, 1]
                nois_batch_data[folder, idx, :, :, 2] = image_noise_aug[:, :, 2]

            image[:, :, 0] = (image [:, :, 0] - np.min(image[:, :, 0]))/(np.max(image[:, :, 0]) - np.min(image[:, :, 0]))
            image[:, :, 1] = (image [:, :, 1] - np.min(image[:, :, 1]))/(np.max(image[:, :, 1]) - np.min(image[:, :, 1]))
            image[:, :, 2] = (image [:, :, 2] - np.min(image[:, :, 2]))/(np.max(image[:, :, 2]) - np.min(image[:, :, 2]))

            if(aug_aff):
                shear_val = random.uniform(0.0, -0.5)
                tf = AffineTransform(shear = shear_val)
                img_aff = transform.warp(image, tf, order=1, preserve_range=True, mode='wrap')
                aug_batch_data[folder, idx, :, :, 0] = img_aff[:, :, 0]
                aug_batch_data[folder, idx, :, :, 1] = img_aff[:, :, 1]
                aug_batch_data[folder, idx, :, :, 2] = img_aff[:, :, 2]
                
            batch_data[folder,idx,:,:,0] = image [:, :, 0]
            batch_data[folder,idx,:,:,1] = image [:, :, 1]
            batch_data[folder,idx,:,:,2] = image [:, :, 2]


            batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            if aug_aff:
                aug_batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            if aug_nois:
                nois_batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            if aug_edg:
                edg_batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1


                
        if aug_aff:
            batch_data = np.append(batch_data, aug_batch_data, axis = 0)
            batch_labels = np.append(batch_labels, aug_batch_labels, axis = 0) 

        if aug_nois:
            batch_data = np.append(batch_data, nois_batch_data, axis = 0)
            batch_labels = np.append(batch_labels, nois_batch_labels, axis = 0) 

        if aug_edg:
            batch_data = np.append(batch_data, edg_batch_data, axis = 0)
            batch_labels = np.append(batch_labels, edg_batch_labels, axis = 0) 

        print(batch_data.shape, batch_labels.shape)     
        yield batch_data, batch_labels                                                            #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = '/Users/sharadchandra/Hand_Gestures'               # '/notebooks/storage/Final_data/Collated_training/train'
val_path = '/Users/sharadchandra/Hand_Gestures/val'                   # '/notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15                                                          # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [9]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

In [10]:
from keras import applications

In [11]:
import tensorflow as tf

if(cnn == 0):
    input_shape = (96, 96, 3)

    mobv2_model = applications.MobileNetV2(input_shape=input_shape, weights = 'imagenet', include_top=False)

    mobv2_model.trainable = False
#     num_layers = len(mobv2_model.layers)     
#     print(num_layers)
    
#     layer_split = 4*num_layers//5
#     print(layer_split)
#     for layer in range(layer_split):
#         mobv2_model.layers[layer].trainable = False
#     for layer in range(layer_split, num_layers):
#         mobv2_model.layers[layer].trainable = True
        
    cnn_model = Sequential([
        mobv2_model,
        keras.layers.Flatten(),
        keras.layers.Dense(64),
#        keras.layers.BatchNormalization(),
#        keras.layers.Dropout(0.25)
    ])
    cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 11520)             0         
                                                                 
 dense (Dense)               (None, 64)                737344    
                                                                 
Total params: 2,995,328
Trainable params: 737,344
Non-trainable params: 2,257,984
_________________________________________________________________


In [12]:
# from keras.layers import Lambda
from keras import regularizers

def rnn_gru(input_dims, n_classes, cnn_model):
    model = Sequential()
    model.add(TimeDistributed(cnn_model, input_shape = input_dims))
#    model.add(GRU(64, input_shape = input_dims, return_sequences=False, kernel_regularizer = regularizers.l2(1e-02), dropout = 0.25))
#    model.add(GRU(64, input_shape = input_dims, return_sequences=False, dropout = 0.25))
    model.add(GRU(64, input_shape = input_dims, return_sequences=False))

    model.add(Dense(64, activation = 'relu'))
#    model.add(BatchNormalization())
#    model.add(Dropout(0.25))
    model.add(Dense(n_classes, activation='softmax'))
    return model

In [13]:
if(cnn == 0):
    if(img_nums == 30):
        input_dimens = (30, 96, 96, 3)
    else:
        input_dimens = (15, 96, 96, 3)
    model = rnn_gru(input_dims = input_dimens, n_classes = 5, cnn_model = cnn_model)

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [14]:
if(cnn == 0):
    #optimiser = keras.optimizers.Adam(learning_rate = 0.01)                            #write your optimizer
    #optimiser = 'Adam'
    #model.compile(optimizer=keras.optimizers.Adam(lr = 0.1), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    #model.compile(optimizer=tf.train.AdamOptimizer(0.01), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    optimiser = 'Adam'
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


    print (model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 15, 64)           2995328   
 ibuted)                                                         
                                                                 
 gru (GRU)                   (None, 64)                24960     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 3,024,773
Trainable params: 766,789
Non-trainable params: 2,257,984
_________________________________________________________________
None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [15]:
if(cnn == 0):
    aug_aff = 0
    aug_edg = 0
    aug_nois = 0
    train_generator = generator('/Users/sharadchandra/Hand_Gestures/train', train_doc, batch_size, train_len, aug_aff = 1, aug_edg = 0, aug_nois = 1)
    val_generator = generator('/Users/sharadchandra/Hand_Gestures/val', val_doc, batch_size, val_len, aug_aff = 0, aug_edg = 0, aug_nois = 0)
    print(train_generator)
    print(val_generator)

<generator object generator at 0x29a917920>
<generator object generator at 0x29a917a00>


In [16]:
if(cnn == 0):
    model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
    if not os.path.exists(model_name):
        os.mkdir(model_name)

    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)     # write the REducelronplateau code here
    callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [17]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print(f'Steps per epoch = {steps_per_epoch}, validation_steps = {validation_steps}')

Steps per epoch = 8, validation_steps = 2


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [26]:
if(cnn == 0):
    model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/var/folders/mz/vg384l6j0h542rj14nk8lhkh0000gn/T/ipykernel_5143/3180245321.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


StopIteration: 

In [18]:
from tensorflow import keras
m2 = keras.models.load_model('/Users/sharadchandra/Hand_Gestures/model-00015-0.38187-0.84163-0.28922-0.87000.h5')#epoch_loss-train_accuracytest_losstest_accuracy

In [19]:
import matplotlib.pyplot as plt

In [34]:
import shutil
import os
source_path = "/Users/sharadchandra/Hand_Gestures/train"
mnp = 0
flag_2 = 0
exit = 0


while(True):

    directory = 'imgs3'
    location = "/Users/sharadchandra/Hand_Gestures/train"
    # Parent Directory path
    parent_dir = location
    dir = "imgs3"
    # Path
    path2 = os.path.join(parent_dir, directory)
    path = os.path.join(location, dir)
    # Create the directory
    # 'GeeksForGeeks' in
    # '/home / User / Documents'
    os.mkdir(path)


    import cv2
    from time import *

    # Opens the inbuilt camera of laptop to capture video.
    cap = cv2.VideoCapture(0)
    i = 0
    if(mnp==0):
        sleep(2)
    mnp+=1

    while(cap.isOpened()):
        ret, frame = cap.read()
        sleep(0.01)
        # This condition prevents from infinite looping
        # incase video ends.
        if ret == False:
            break
        #cv2.imshow('frame', frame)
        # Save Frame by Frame into disk using imwrite method
        #cv2.imwrite('Frame'+str(i)+'.jpg', frame)
        path = '/Users/sharadchandra/Hand_Gestures/train/imgs3'
        cv2.imwrite(os.path.join(path ,  str(i) + '.png'), frame)
        i += 1
        if (i == 45):
            break

    cap.release()
    cv2.destroyAllWindows()





    def pred_creator(source_path, folder_list, batch_size, lenth ):
        print( 'Source path = ', source_path, '; batch size =', batch_size)

        l = []
        i = 0

        if(img_nums == 30):
            img_idx = range(0, 30)                                                                     #create a list of image numbers you want to use for a particular video
        else:
            img_idx = range(0, 30, 2)

        t = np.random.permutation(folder_list)
        #t = folder_list
        num_batches = lenth // batch_size                                                     # calculate the number of batches
        num_batches = 1
        num_imgs = lenth - num_batches*batch_size
        for batch in range(num_batches):                                                          # we iterate over the number of batches
            x = len(img_idx)
            y = 96
            z = 96
            batch_data = np.zeros((batch_size,x,y,z,3))                                           # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
    #           batch_labels = np.zeros((batch_size,5))                                               # batch_labels is the one hot representation of the output
            batch_labels = np.zeros((30, 5))                                               # batch_labels is the one hot representation of the output


    # WIN_20180925_17_44_26_Pro_Right_Swipe_new;Right_Swipe_new;1


            for folder in range(batch_size):                                                      # iterate over the batch_size
    #            imgs = os.listdir(source_path+'/'+ 'WIN_20180925_17_08_43_Pro_Left_Swipe_new')
                #imgs = os.listdir(source_path+'/'+ 'WIN_20180925_17_44_26_Pro_Right_Swipe_new')
                print(source_path + '/' + 'imgs3')
                imgs = os.listdir('/Users/sharadchandra/Hand_Gestures/train'+'/'+ 'imgs3')
                #imgs = sorted(imgs)
                #print(imgs)                                                                       # read all the images in the folder
                for idx,item in enumerate(img_idx):                                               #  Iterate over the frames/images of a folder to read them in
    #                    print("This Img Path is: ")
    #                    print(source_path+'/'+ 'WIN_20180925_17_08_43_Pro_Left_Swipe_new'+'/'+imgs[item])
    #                image = imread(source_path+'/'+ 'WIN_20180925_17_44_26_Pro_Right_Swipe_new'+'/'+imgs[item]).astype(np.float32)
                    image = imread('/Users/sharadchandra/Hand_Gestures/train'+'/'+ 'imgs3'+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    image[:, :, 0] = (image [:, :, 0] - np.min(image[:, :, 0]))/(np.max(image[:, :, 0]) - np.min(image[:, :, 0]))
                    image[:, :, 1] = (image [:, :, 1] - np.min(image[:, :, 1]))/(np.max(image[:, :, 1]) - np.min(image[:, :, 1]))
                    image[:, :, 2] = (image [:, :, 2] - np.min(image[:, :, 2]))/(np.max(image[:, :, 2]) - np.min(image[:, :, 2]))


                    image = resize(image = image, output_shape=(y, z))

                    batch_data[folder,idx,:,:,0] = image[:, :, 0]       #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:, :, 1]       #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:, :, 2]       #normalise and feed in the image

    #                l.append(image)
    #               batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels[folder, 0]= 1
        print("SHAPE = " , image.shape)
    #        l.append(batch_labels)
            #return l
    #        l1 = np.array(l)
        print("***************** batch_data.shape = ***************", batch_data.shape)
        yield batch_data, batch_labels
    #        yield l1, batch_labels
    #        return l1, batch_labels
    #        return batch_data, batch_labels

    p2 = pred_creator(train_path, train_doc, 1, 15)
    pred_mat1 = m2.predict(p2)
    d = ['Left Swipe' , 'Right Swipe' , 'Stop' , 'Thumbs Down' , 'Thumbs Up']
    # 0: Left Swipe, 1: Right Swipe, 2: Stop, 3: Thumbs Down, 4: Thumbs Up

    l_pred = pred_mat1.tolist()
    l_pred = l_pred[0]
    v = 0
    flag_2 = 0
    print(l_pred)
    for i in range(len(l_pred)):
        if l_pred[i] == max(l_pred):
            print (d[i])
            v = i
            break
    flag = 0
    for i in l_pred:
        if i>0.5:
            flag = 1
    if(flag):
        from pynput.keyboard import Key, Controller
        from time import *

        keyboard = Controller()

        def spacebar():
            keyboard.press(Key.space)
            keyboard.release(Key.space)

        def right():
            keyboard.press(Key.right)
            keyboard.release(Key.right)

        def left():
            keyboard.press(Key.left)
            keyboard.release(Key.left)

        def up():
            keyboard.press(Key.up)
            keyboard.release(Key.up)

        def down():
            keyboard.press(Key.down)
            keyboard.release(Key.down)
        def escp():
            keyboard.press(Key.esc)
            keyboard.press(Key.esc)
        def enter():
            keyboard.press(Key.f5)
            keyboard.press(Key.f5)

        x = v
        print(x)
        if x == 0:
            sleep(0.5)
            left()
        if x == 1:
            sleep(0.5)
            right()
        if x == 2:
            sleep(5)
            #left()
        if x == 3:
            sleep(0.5)
            flag_2 = 1
            break
        if x == 4:
            sleep(0.5)
            if (exit == 0):
                escp()
                exit += 1
            else:
                enter()
                exit-=1
                
              

    # Python program to demonstrate
    # shutil.rmtree()
    



    # location
    location = "/Users/sharadchandra/Hand_Gestures/train"

    # directory
    dir = "imgs3"

    # path
    path = os.path.join(location, dir)

    # removing directory
    shutil.rmtree(path)
    if(flag_2):
        break

if(flag_2 == 1):
    dir = "imgs3"

    # path
    path = os.path.join(location, dir)

    # removing directory
    shutil.rmtree(path)

Source path =  /Users/sharadchandra/Hand_Gestures ; batch size = 1
/Users/sharadchandra/Hand_Gestures/imgs3
SHAPE =  (96, 96, 3)
***************** batch_data.shape = *************** (1, 15, 96, 96, 3)
1/1 [==============================] - 0s 49ms/step
[0.42843928933143616, 0.009935986250638962, 0.11888343840837479, 0.10312936455011368, 0.3396119773387909]
Left Swipe
Source path =  /Users/sharadchandra/Hand_Gestures ; batch size = 1
/Users/sharadchandra/Hand_Gestures/imgs3
SHAPE =  (96, 96, 3)
***************** batch_data.shape = *************** (1, 15, 96, 96, 3)
1/1 [==============================] - 0s 47ms/step
[0.15442323684692383, 0.03578585386276245, 0.44656988978385925, 0.06586601585149765, 0.2973549962043762]
Stop
Source path =  /Users/sharadchandra/Hand_Gestures ; batch size = 1
/Users/sharadchandra/Hand_Gestures/imgs3
SHAPE =  (96, 96, 3)
***************** batch_data.shape = *************** (1, 15, 96, 96, 3)
1/1 [==============================] - 0s 47ms/step
[0.20718589425

KeyboardInterrupt: 

In [ ]:
! pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#pip install pynput

Note: you may need to restart the kernel to use updated packages.


In [ ]:
location = "/Users/sharadchandra/Hand_Gestures/train"

    # directory
dir = "imgs3"

    # path
path = os.path.join(location, dir)

    # removing directory
shutil.rmtree(path)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/sharadchandra/Hand_Gestures/train/imgs3'